In [ ]:
import geemap
import ee
import os

In [ ]:
Map = geemap.Map()
Map

In [ ]:
os.chdir(r'E:/Work/Sample_points/')

roi_path = r'sample_point_global.shp'
points = geemap.shp_to_ee(roi_path)
Map.addLayer(points, {}, 'points')

In [ ]:
import pandas as pd

df = pd.DataFrame({'ID':points.aggregate_array("ID").getInfo(),
                  'X':points.aggregate_array("X").getInfo(),
                 'Y':points.aggregate_array("Y").getInfo()})
df

In [ ]:
for year in [2019, 2021]:
    data = df.copy()
    image = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
                .filterDate(str(year), str(year+1)).mean()

    points_t1 = image.sampleRegions(points, 
                                    geometries=True, 
                                    scale=image.projection().nominalScale())

    for i, key in enumerate(image.bandNames().getInfo()):
        data.insert(3+i, key, points_t1.aggregate_array(key).getInfo())
    
    data.to_csv(f'GLDAS_{year}.csv', index=False, header=True)

In [ ]:
for year in [2019, 2021]:
    data = df.copy()
    images = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterDate(str(year), str(year+1))
    image = images.mean()
    prcp = images.select('pr').sum()
    points_t1 = image.sampleRegions(points, 
                                    geometries=True, 
                                    scale=image.projection().nominalScale())
    points_t2 = prcp.sampleRegions(points, 
                                    geometries=True, 
                                    scale=prcp.projection().nominalScale())
    for i, key in enumerate(image.bandNames().getInfo()):
        data.insert(3+i, key, points_t1.aggregate_array(key).getInfo())
    data['pr'] = points_t2.aggregate_array('pr').getInfo()
    data.to_csv(f'IDAHO_EPSCOR_{year}.csv', index=False, header=True)

In [ ]:
for year in [2019, 2021]:
    data = df.copy()
    images = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(str(year), str(year+1))
    image = images.mean()
    prcp = images.select('total_precipitation').sum()
    points_t1 = image.sampleRegions(points, 
                                    geometries=True, 
                                    scale=image.projection().nominalScale())
    points_t2 = prcp.sampleRegions(points, 
                                    geometries=True, 
                                    scale=prcp.projection().nominalScale())
    for i, key in enumerate(image.bandNames().getInfo()):
        data.insert(3+i, key, points_t1.aggregate_array(key).getInfo())
    data['total_precipitation'] = points_t2.aggregate_array('total_precipitation').getInfo()
    data.to_csv(f'ERA5_{year}.csv', index=False, header=True)